In [1]:
import copy
from pathlib import Path
import warnings
import random

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

import matplotlib.pyplot as plt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
input_length = 336
output_length = 168

show_graph = False

In [3]:
# Group 3
model_paths = [ "Save_File/Group3_4/A1.ckpt",
                "Save_File/Group3_4/A2.ckpt",
                "Save_File/Group3_4/A3.ckpt",
                "Save_File/Group3_4/A4.ckpt",
                "Save_File/Group3_4/A5.ckpt",
                "Save_File/Group3_4/A6.ckpt",
                "Save_File/Group3_4/A7.ckpt",
                "Save_File/Group3_4/B1.ckpt",
                "Save_File/Group3_4/B2.ckpt",
                "Save_File/Group3_4/B3.ckpt",
                "Save_File/Group3_4/B4.ckpt",
                "Save_File/Group3_4/B5.ckpt",
                "Save_File/Group3_4/C1.ckpt",
                "Save_File/Group3_4/C2.ckpt",
                "Save_File/Group3_4/C3.ckpt",
                "Save_File/Group3_4/C4.ckpt",
                "Save_File/Group3_4/D1.ckpt",
                "Save_File/Group3_4/D2.ckpt",
                "Save_File/Group3_4/D3.ckpt",
                "Save_File/Group3_4/D4.ckpt",
                "Save_File/Group3_4/D5.ckpt",
                "Save_File/Group3_4/D6.ckpt",
                "Save_File/Group3_4/E1.ckpt",
                "Save_File/Group3_4/E2.ckpt",
                "Save_File/Group3_4/E3.ckpt",
                "Save_File/Group3_4/E4.ckpt",
                "Save_File/Group3_4/E5.ckpt",
                "Save_File/Group3_4/E6.ckpt",]

group_name = [
    "A1", "A2", "A3", "A4", "A5", "A6", "A7",
    "B1", "B2", "B3", "B4", "B5",
    "C1", "C2", "C3", "C4", 
    "D1", "D2", "D3", "D4", "D5", "D6", 
    "E1", "E2", "E3", "E4", "E5", "E6"
]

group = [
    [4],[11, 12],[34],[40],[10],[42],[41]
    ,[6, 8, 13, 17, 26, 48, 53, 55, 56],[7, 18],[27, 57],[35, 46, 47],[25]
    ,[1, 31],[9, 32],[3],[33]
    ,[2, 14, 22, 37, 44, 52, 54],[15],[38, 58, 43],[29, 39],[45],[23]
    ,[5],[16, 24],[19, 20, 21, 49, 50, 51],[28, 36, 60],[59], [30]
]

data_path = "./Refined_Data/Grouped_Data/Input_Data2.csv"

In [4]:
dataframes = pd.read_csv(data_path, parse_dates = ["date_time"])

dataframes['num']     =   dataframes['num'].apply(str)
dataframes['day_of_Week']     =   dataframes['day_of_Week'].apply(str)
dataframes['day_of_month']    =   dataframes['day_of_month'].apply(str)
dataframes['24Hour']  =   dataframes['24Hour'].apply(str)
dataframes['holiday'] =   dataframes['holiday'].apply(str)
dataframes['Weekend'] =   dataframes['Weekend'].apply(str)
dataframes['energy_group'] = dataframes['energy_group'].apply(str)
dataframes['hour_cat']=   dataframes['hour_cat'].apply(str)

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

all_smape = []
part_smape = [[] for i in range(len(group_name))]

In [5]:
random_choice = 3 # random.randrange(3,12)

start_point = -168 * random_choice - 1
duration = 168

In [6]:
for i in range(len(model_paths)):

    print(f"now group : {group_name[i]}")

    for building in group[i]:
        now_building_dataframe = dataframes.loc[dataframes["num"] == str(building)].copy()
        now_building_dataframe = now_building_dataframe[start_point:start_point+input_length+duration]

        now_best_tft = TemporalFusionTransformer.load_from_checkpoint(model_paths[i])

        original = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()
        prediction = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()  

        total_cycle = ((len(now_building_dataframe) - input_length - output_length) // output_length) + 1

        for k in range(total_cycle):
            now_start_point = k*output_length

            encoder_data = now_building_dataframe.iloc[now_start_point:now_start_point+input_length].copy()
            decoder_data = now_building_dataframe.iloc[now_start_point+input_length:now_start_point+input_length+output_length].copy()
            new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

            raw_predictions = now_best_tft.predict(new_prediction_data, mode="prediction").numpy().tolist()[0]
            originals = now_building_dataframe.iloc[now_start_point+input_length : now_start_point+input_length+output_length]["kWH"].tolist()

            prediction.extend(raw_predictions)
            original.extend(originals)

            # for next step, change dataframe's original value to predicted value
            for p in range(output_length):
                now_building_dataframe.iloc[now_start_point+input_length+p, now_building_dataframe.columns.get_loc("kWH")] = raw_predictions[p]
            
        if show_graph:
            plt.rcParams["figure.figsize"] = (17,5)
            fig = plt.figure()
            graph = fig.add_subplot(1, 1, 1)
            graph.plot(prediction, color='blue')
            graph.plot(original, color='red')
            plt.show()

        smape_loss = smape(np.array(prediction[168:]), np.array(original[168:]))
        print(f"building : {building}, score : {smape_loss}")
        all_smape.append(smape_loss)
        part_smape[i].append(smape_loss)

        # break
    # break

now group : A1
building : 4, score : 10.857579665529213
now group : A2
building : 11, score : 4.316399789682509
building : 12, score : 4.6548944009209965
now group : A3
building : 34, score : 7.005717259771462
now group : A4
building : 40, score : 7.915903510096563
now group : A5
building : 10, score : 7.453955992029312
now group : A6
building : 42, score : 8.790247665482708
now group : A7
building : 41, score : 6.177566354154075
now group : B1
building : 6, score : 2.653645287902317
building : 8, score : 3.141082161943811
building : 13, score : 3.034433010526445
building : 17, score : 2.923733259510167
building : 26, score : 2.12795405946186
building : 48, score : 5.641797546221138
building : 53, score : 1.987878645143017
building : 55, score : 2.6534533631755592
building : 56, score : 4.414731925319988
now group : B2
building : 7, score : 4.090108057311967
building : 18, score : 4.145287036157915
now group : B3
building : 27, score : 6.352092756719947
building : 57, score : 3.2446398

In [7]:
print(f"selected time slice : {random_choice}")
print(f"totoal score : {np.mean(np.array(all_smape))}")
for i in range(len(group_name)):
    print(f"{group_name[i]} score ({len(group[i])}) : {np.mean(np.array(part_smape[i]))}")

selected time slice : 3
totoal score : 3.6092841473325703
A1 score (1) : 10.857579665529213
A2 score (2) : 4.485647095301752
A3 score (1) : 7.005717259771462
A4 score (1) : 7.915903510096563
A5 score (1) : 7.453955992029312
A6 score (1) : 8.790247665482708
A7 score (1) : 6.177566354154075
B1 score (9) : 3.175412139911589
B2 score (2) : 4.117697546734941
B3 score (2) : 4.798366303069614
B4 score (3) : 3.906132343830412
B5 score (1) : 4.7594415837574555
C1 score (2) : 0.44095472800350044
C2 score (2) : 0.6548382948665856
C3 score (1) : 0.46539246531765854
C4 score (1) : 0.4194462566045745
D1 score (7) : 2.24503760665185
D2 score (1) : 4.306090448119504
D3 score (3) : 2.5702510684457676
D4 score (2) : 5.547011456881487
D5 score (1) : 2.529252876129431
D6 score (1) : 5.709417611129836
E1 score (1) : 4.81752558487628
E2 score (2) : 2.0328244183823654
E3 score (6) : 3.0446529725552876
E4 score (3) : 2.5256904207489517
E5 score (1) : 7.8483765963243135
E6 score (1) : 3.778343443977009
